# Prediksi Keselamatan Penumpang Titanic Menggunakan Logistic Regression

## Anggota Kelompok

* Hafizh Marfiansyah Putra - 103012300201
* Azzahra Indah - 103012300238


## Pendahuluan dan Paparan Data

Tragedi tenggelamnya kapal **RMS Titanic** pada tahun 1912 merupakan salah satu peristiwa paling terkenal dalam sejarah pelayaran. Dari kejadian tersebut, tidak semua penumpang memiliki peluang yang sama untuk selamat. Faktor seperti kelas penumpang, jenis kelamin, usia, serta kondisi keluarga diduga memengaruhi tingkat keselamatan penumpang.

Pada tugas besar ini, kami membangun sebuah **model Artificial Intelligence menggunakan algoritma Logistic Regression** untuk memprediksi apakah seorang penumpang Titanic **selamat (Survived)** atau **tidak selamat**, berdasarkan beberapa atribut penumpang yang tersedia dalam dataset.

## Dataset dan Sumber Data

Dataset yang digunakan adalah **Titanic Dataset**, yang berisi data penumpang kapal Titanic beserta status keselamatannya. Dataset ini banyak digunakan sebagai studi kasus dalam machine learning karena memiliki struktur data yang jelas dan relevan untuk klasifikasi biner.

Variabel yang digunakan dalam penelitian ini adalah:

* **Survived**: Status keselamatan penumpang (target)
* **Pclass**: Kelas penumpang
* **Sex**: Jenis kelamin
* **Age**: Usia penumpang
* **SibSp**: Jumlah saudara atau pasangan di kapal
* **Parch**: Jumlah orang tua atau anak di kapal
* **Fare**: Harga tiket
* **Embarked**: Pelabuhan keberangkatan

## Pre-processing Dataset

Sebelum membangun model, dataset melalui beberapa tahapan pre-processing, antara lain:

* Pemilihan fitur yang relevan sesuai variabel yang digunakan
* Penanganan data kategorikal seperti **Sex** dan **Embarked**
* Penanganan data kosong atau tidak valid
* Penyesuaian format data agar dapat diproses oleh model Logistic Regression

Tahapan ini bertujuan untuk meningkatkan kualitas data dan performa model yang dibangun.

## Pembagian Data Training, Validasi, dan Testing

Dataset dibagi menjadi tiga bagian untuk memastikan model dapat dilatih dan dievaluasi dengan baik:

* **60% data** digunakan sebagai **training set** untuk melatih model
* **20% data** digunakan sebagai **validation set** untuk mengevaluasi dan menyetel model
* **20% data** digunakan sebagai **testing set** untuk mengukur performa akhir model

Pembagian data dilakukan secara acak dengan menggunakan seed tertentu agar hasil eksperimen dapat direproduksi.


In [36]:
import random
# ===============================
# TRAIN / VALIDATION / TEST SPLIT (60 / 20 / 20)
# ===============================
random.seed(42)

indices = list(range(len(X)))
random.shuffle(indices)

n_total = len(indices)
train_end = int(0.6 * n_total)
val_end   = int(0.8 * n_total)

train_idx = indices[:train_end]
val_idx   = indices[train_end:val_end]
test_idx  = indices[val_end:]

X_train = [X[i] for i in train_idx]
y_train = [y[i] for i in train_idx]

X_val = [X[i] for i in val_idx]
y_val = [y[i] for i in val_idx]

X_test = [X[i] for i in test_idx]
y_test = [y[i] for i in test_idx]

print("\nDataset split:")
print("Training   :", len(X_train))
print("Validation :", len(X_val))
print("Testing    :", len(X_test))



Dataset split:
Training   : 427
Validation : 142
Testing    : 143


In [37]:
# ===============================
# HELPER FUNCTIONS
# ===============================

def find_mean(dataset):
    return sum(dataset) / len(dataset)

def find_standard_deviation(dataset, mean):
    return (sum((x - mean) ** 2 for x in dataset) / len(dataset)) ** 0.5


# ===============================
# COMPUTE MEANS & STDS (TRAIN ONLY)
# ===============================

feature_means = []
feature_stds = []

for feature_idx in range(n_features):
    column_values = [row[feature_idx] for row in X_train]

    mean = find_mean(column_values)
    std = find_standard_deviation(column_values, mean)

    if std == 0:
        std = 1.0

    feature_means.append(mean)
    feature_stds.append(std)

print("Means:", feature_means)
print("Stds :", feature_stds)


Means: [2.281030444964871, 0.36065573770491804, 29.060117096018736, 0.4637002341920375, 0.4168618266978923, 32.8852355971897, 0.24824355971896955]
Stds : [0.8014570259300761, 0.48019077101235447, 14.155581937394468, 0.9336129533561637, 0.8708102701367192, 54.02957558215128, 0.5159883598658702]


## Metode dan Eksperimen

### Metode yang Digunakan

Pada penelitian ini, metode yang digunakan adalah **Logistic Regression**, yaitu salah satu algoritma klasifikasi biner yang memodelkan hubungan antara variabel input dan probabilitas suatu kelas. Logistic Regression digunakan untuk menghitung **probabilitas seorang penumpang Titanic selamat atau tidak selamat**, dengan memanfaatkan fungsi sigmoid sebagai fungsi aktivasi.

Model dibangun dari awal tanpa menggunakan library machine learning eksternal. Proses pelatihan dilakukan menggunakan **gradient descent** dengan regularisasi L2 (ridge) untuk mengurangi risiko overfitting. Parameter model yang dipelajari meliputi bobot (weight) untuk setiap fitur dan sebuah bias.

Fungsi sigmoid digunakan untuk mengubah hasil perhitungan linear menjadi nilai probabilitas antara 0 dan 1. Nilai probabilitas tersebut kemudian dikonversi menjadi kelas prediksi dengan threshold 0.5.

### Pengukuran Kinerja Model

Kinerja model dievaluasi menggunakan **metrik akurasi (accuracy)**. Akurasi mengukur seberapa besar proporsi prediksi yang benar dibandingkan dengan keseluruhan data.

Evaluasi dilakukan pada tiga subset data:

* **Training set** untuk melatih model
* **Validation set** untuk memilih hyperparameter terbaik
* **Testing set** untuk mengukur performa akhir model

Penggunaan data validasi bertujuan agar proses pemilihan hyperparameter tidak bias terhadap data uji, sehingga hasil evaluasi lebih objektif.

### Hyperparameter dan Hyperparameter Tuning

Beberapa hyperparameter utama yang digunakan dalam model Logistic Regression adalah:

* **Learning rate**, yang mengatur besar langkah pembaruan parameter
* **Lambda**, sebagai parameter regularisasi untuk mengontrol kompleksitas model
* **Jumlah epoch**, yang menentukan berapa kali proses pelatihan dilakukan terhadap seluruh data training

Untuk memperoleh kombinasi hyperparameter yang optimal, digunakan metode **Genetic Algorithm**. Genetic Algorithm bekerja dengan meniru proses evolusi biologis, yaitu melalui tahapan inisialisasi populasi, evaluasi fitness, seleksi, crossover, dan mutasi.

Setiap individu dalam populasi merepresentasikan satu kombinasi hyperparameter. Nilai **fitness** ditentukan berdasarkan akurasi model pada data validasi. Individu dengan performa terbaik akan dipertahankan dan dikombinasikan untuk menghasilkan generasi berikutnya.

Proses ini dijalankan selama beberapa generasi hingga diperoleh kombinasi hyperparameter dengan performa validasi terbaik. Hyperparameter hasil Genetic Algorithm kemudian digunakan untuk melatih model akhir sebelum dilakukan pengujian pada data testing.

In [38]:
import math
import random

# ===============================
# HYPERPARAMETERS
# ===============================
learning_rate = 0.004294434790043332
lambda_ = 0.007787368581333586
epochs = 9001

# ===============================
# DATA SIZES
# ===============================
n_samples = len(X_train)
n_features = len(X_train[0])

# ===============================
# INITIALIZE PARAMETERS
# ===============================
weight = [random.uniform(-0.01, 0.01) for _ in range(n_features)]
b = 0.0


# ===============================
# MODEL FUNCTIONS
# ===============================
def sigmoid(z):
    if z >= 0:
        return 1 / (1 + math.exp(-z))
    else:
        ez = math.exp(z)
        return ez / (1 + ez)


def find_weighted_sum_for_row(row):
    return sum(weight[i] * row[i] for i in range(n_features)) + b


# ===============================
# TRAINING
# ===============================
def train_logistic_regression(X_train, y_train, lr, lambda_, epochs):
    n_samples = len(X_train)
    n_features = len(X_train[0])

    # initialize parameters
    weight = [random.uniform(-0.01, 0.01) for _ in range(n_features)]
    b = 0.0

    for _ in range(epochs):
        preds = [
            sigmoid(sum(weight[i] * X_train[j][i] for i in range(n_features)) + b)
            for j in range(n_samples)
        ]

        # update weights
        for i in range(n_features):
            grad = sum((preds[j] - y_train[j]) * X_train[j][i] for j in range(n_samples)) / n_samples
            grad += lambda_ * weight[i]
            weight[i] -= lr * grad

        # update bias
        b_grad = sum(preds[j] - y_train[j] for j in range(n_samples)) / n_samples
        b -= lr * b_grad

    return weight, b


def train_model(epochs):
    global weight, b
    weight, b = train_logistic_regression(X_train, y_train, learning_rate, lambda_, epochs)
    print(f"Training completed.")

def accuracy(X, y, weight, b):
    correct = 0
    for i in range(len(X)):
        z = sum(weight[j] * X[i][j] for j in range(len(weight))) + b
        pred = 1 if sigmoid(z) >= 0.5 else 0
        if pred == y[i]:
            correct += 1
    return correct / len(X)


# ===============================
# TRAIN
# ===============================
train_model(epochs)

print("\nTrained weights:", weight)
print("Trained bias:", b)


# ===============================
# TEST ON TEST SET
# ===============================
print("\nTest predictions (first 5):")
for i in range(5):
    p = sigmoid(find_weighted_sum_for_row(X_test[i]))
    print(f"Sample {i}: probability={p:.4f}, predicted={1 if p>=0.5 else 0}, actual={y_test[i]}")


Training completed.

Trained weights: [-0.5032533704344994, 1.8484911245378948, -0.027376731108499468, -0.24131936704292176, 0.05179414316065028, 0.0033397057643350936, -0.009711819698345928]
Trained bias: 0.4766514618701213

Test predictions (first 5):
Sample 0: probability=0.5525, predicted=1, actual=1
Sample 1: probability=0.1784, predicted=0, actual=0
Sample 2: probability=0.6899, predicted=1, actual=1
Sample 3: probability=0.2560, predicted=0, actual=0
Sample 4: probability=0.1826, predicted=0, actual=1


In [39]:
# ===============================
# GENETIC ALGORITHM
# ===============================

POP_SIZE = 8
GENERATIONS = 10
MUTATION_RATE = 0.2


def random_individual():
    return {
        "lr": random.uniform(0.001, 0.05),
        "lambda": random.uniform(0.00001, 0.01),
        "epochs": random.randint(3000, 15000)
    }


def fitness(individual):
    weight, b = train_logistic_regression(X_train, y_train,individual["lr"],individual["lambda"],individual["epochs"]
    )
    return accuracy(X_val, y_val, weight, b)


# initialize population
population = [random_individual() for _ in range(POP_SIZE)]

best_individual = None
best_score = 0

for gen in range(GENERATIONS):
    print(f"\nGeneration {gen}")

    scored = []
    for ind in population:
        score = fitness(ind)
        scored.append((score, ind))
        print(f"  {ind} → val acc = {score:.4f}")

    scored.sort(reverse=True, key=lambda x: x[0])

    if scored[0][0] > best_score:
        best_score = scored[0][0]
        best_individual = scored[0][1]

    # selection (top 50%)
    survivors = [ind for _, ind in scored[:POP_SIZE // 2]]

    # crossover + mutation
    new_population = survivors.copy()

    while len(new_population) < POP_SIZE:
        parent1 = random.choice(survivors)
        parent2 = random.choice(survivors)

        child = {
            "lr": random.choice([parent1["lr"], parent2["lr"]]),
            "lambda": random.choice([parent1["lambda"], parent2["lambda"]]),
            "epochs": random.choice([parent1["epochs"], parent2["epochs"]]),
        }

        # mutation
        if random.random() < MUTATION_RATE:
            child["lr"] *= random.uniform(0.8, 1.2)
        if random.random() < MUTATION_RATE:
            child["lambda"] *= random.uniform(0.8, 1.2)

        new_population.append(child)

    population = new_population


print("\nBEST HYPERPARAMETERS FOUND:")
print(best_individual)
print("Validation accuracy:", best_score)




Generation 0
  {'lr': 0.016240907060331395, 'lambda': 0.005791060477767343, 'epochs': 8111} → val acc = 0.4366
  {'lr': 0.029214039692208295, 'lambda': 0.009081181248963972, 'epochs': 9149} → val acc = 0.7324
  {'lr': 0.020436067634314742, 'lambda': 0.007152996903444189, 'epochs': 4245} → val acc = 0.7394
  {'lr': 0.03001279520124276, 'lambda': 0.00830741062146538, 'epochs': 6978} → val acc = 0.6831
  {'lr': 0.0059931639668528505, 'lambda': 0.007727084040662728, 'epochs': 14210} → val acc = 0.7394
  {'lr': 0.030420169126025364, 'lambda': 0.0012193401000224781, 'epochs': 12272} → val acc = 0.7465
  {'lr': 0.03934913197168996, 'lambda': 0.003478565615431406, 'epochs': 10018} → val acc = 0.7183
  {'lr': 0.03341454569368044, 'lambda': 0.0006988777978312089, 'epochs': 13609} → val acc = 0.7394

Generation 1
  {'lr': 0.030420169126025364, 'lambda': 0.0012193401000224781, 'epochs': 12272} → val acc = 0.6549
  {'lr': 0.020436067634314742, 'lambda': 0.007152996903444189, 'epochs': 4245} → val 

In [40]:
# ===============================
# MODEL EVALUATION (FROM SCRATCH)
# ===============================

def predict_proba(X):
    """Return probability predictions for input X"""
    return [
        sigmoid(sum(weight[i] * row[i] for i in range(n_features)) + b)
        for row in X
    ]


def predict(X, threshold=0.5):
    """Return class predictions for input X"""
    return [1 if p >= threshold else 0 for p in predict_proba(X)]


def evaluate_model(y_true, y_pred, y_proba=None, set_name="Dataset"):
    """
    Logistic Regression Evaluation (Binary Classification)
    """

    n = len(y_true)

    # ---- Metrics helpers ----
    def accuracy():
        return sum(1 for i in range(n) if y_true[i] == y_pred[i]) / n

    def precision():
        tp = sum(1 for i in range(n) if y_true[i] == 1 and y_pred[i] == 1)
        fp = sum(1 for i in range(n) if y_true[i] == 0 and y_pred[i] == 1)
        return tp / (tp + fp) if (tp + fp) > 0 else 0

    def recall():
        tp = sum(1 for i in range(n) if y_true[i] == 1 and y_pred[i] == 1)
        fn = sum(1 for i in range(n) if y_true[i] == 1 and y_pred[i] == 0)
        return tp / (tp + fn) if (tp + fn) > 0 else 0

    def f1():
        p = precision()
        r = recall()
        return 2 * p * r / (p + r) if (p + r) > 0 else 0

    def confusion_matrix():
        tp = sum(1 for i in range(n) if y_true[i] == 1 and y_pred[i] == 1)
        tn = sum(1 for i in range(n) if y_true[i] == 0 and y_pred[i] == 0)
        fp = sum(1 for i in range(n) if y_true[i] == 0 and y_pred[i] == 1)
        fn = sum(1 for i in range(n) if y_true[i] == 1 and y_pred[i] == 0)
        return tp, tn, fp, fn

    # ---- Compute ----
    acc = accuracy()
    prec = precision()
    rec = recall()
    f1s = f1()
    tp, tn, fp, fn = confusion_matrix()

    # ---- Display ----
    print("\n" + "=" * 60)
    print(f"MODEL EVALUATION — {set_name.upper()}")
    print("=" * 60)

    print("\nMetrics:")
    print(f"  Accuracy : {acc:.4f} ({acc*100:.2f}%)")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall   : {rec:.4f}")
    print(f"  F1-score : {f1s:.4f}")

    print("\nConfusion Matrix:")
    print("                 Predicted")
    print("               |  Died | Survived")
    print(f"Actual Died    |  {tn:<4} | {fp}")
    print(f"Actual Survived|  {fn:<4} | {tp}")

    if y_proba is not None:
        print("\nProbability Statistics:")
        print(f"  Min : {min(y_proba):.4f}")
        print(f"  Max : {max(y_proba):.4f}")
        print(f"  Mean: {sum(y_proba)/len(y_proba):.4f}")

    print("=" * 60)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1s,
        "TP": tp,
        "TN": tn,
        "FP": fp,
        "FN": fn
    }

y_proba = predict_proba(X_test)
y_pred = predict(X_test, threshold=0.5)

results = evaluate_model(
    y_true=y_test,
    y_pred=y_pred,
    y_proba=y_proba,
    set_name="Titanic Test Set"
)



MODEL EVALUATION — TITANIC TEST SET

Metrics:
  Accuracy : 0.7762 (77.62%)
  Precision: 0.7674
  Recall   : 0.6000
  F1-score : 0.6735

Confusion Matrix:
                 Predicted
               |  Died | Survived
Actual Died    |  78   | 10
Actual Survived|  22   | 33

Probability Statistics:
  Min : 0.0460
  Max : 0.8499
  Mean: 0.3263
